# Step 1) Get CSV Data

In [1]:
#Get packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3




## Load Data Set

In [2]:
df_csv = pd.read_csv("FantasyPros_Fantasy_Football_Statistics_WR.csv")
df_csv.head()

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,OWN
0,1.0,Michael Thomas (NO),149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%
1,2.0,Chris Godwin (TB),86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%
2,3.0,Kenny Golladay (DET),65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%
3,4.0,Cooper Kupp (LAR),94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%
4,5.0,Julio Jones (ATL),99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%


In [3]:
#2) Split the player name into player and team
df_csv[['Player','Team']] = df_csv['Player'].str.split('(',expand=True)
df_csv.head(5)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,OWN,Team
0,1.0,Michael Thomas,149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%,NO)
1,2.0,Chris Godwin,86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%,TB)
2,3.0,Kenny Golladay,65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%,DET)
3,4.0,Cooper Kupp,94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%,LAR)
4,5.0,Julio Jones,99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%,ATL)


In [4]:
#3) Remove the trailing bracket from the new Team field
# initializing bad_chars_list 

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

for char in spec_chars:
    df_csv['Team'] = df_csv['Team'].str.replace(char, ' ')
df_csv.head()

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,OWN,Team
0,1.0,Michael Thomas,149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%,NO
1,2.0,Chris Godwin,86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%,TB
2,3.0,Kenny Golladay,65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%,DET
3,4.0,Cooper Kupp,94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%,LAR
4,5.0,Julio Jones,99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%,ATL


In [5]:
#4) Update field names
df_csv.rename(columns = {'YDS':'REC_YDS'}, inplace = True) 
df_csv.rename(columns = {'TD':'REC_TD'}, inplace = True) 
df_csv.rename(columns = {'TD.1':'RUSH_TD'}, inplace = True)
df_csv.head()

,Rank,Player,REC,TGT,REC_YDS,Y/R,LG,20+,REC_TD,ATT,YDS.1,RUSH_TD,FL,G,FPTS,FPTS/G,OWN,Team
0,1.0,Michael Thomas,149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%,NO
1,2.0,Chris Godwin,86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%,TB
2,3.0,Kenny Golladay,65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%,DET
3,4.0,Cooper Kupp,94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%,LAR
4,5.0,Julio Jones,99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%,ATL


In [6]:
#5) Remove Null fields from data
# drop all rows with any NaN and NaT values
df_csv_2019 = df_csv.dropna()
df_csv_2019.head()

,Rank,Player,REC,TGT,REC_YDS,Y/R,LG,20+,REC_TD,ATT,YDS.1,RUSH_TD,FL,G,FPTS,FPTS/G,OWN,Team
0,1.0,Michael Thomas,149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%,NO
1,2.0,Chris Godwin,86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%,TB
2,3.0,Kenny Golladay,65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%,DET
3,4.0,Cooper Kupp,94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%,LAR
4,5.0,Julio Jones,99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%,ATL


In [7]:
#6) Check for Duplicates Duplicates
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = df_csv_2019[df_csv_2019.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [Rank, Player, REC, TGT, REC_YDS, Y/R, LG, 20+, REC_TD, ATT, YDS.1, RUSH_TD, FL, G, FPTS, FPTS/G, OWN, Team]
Index: []


In [8]:
#Connect to SQL Lite Database
conn = sqlite3.connect('DSC540_final_db.sqlite')
#Insert dataframe
df_csv_2019.to_sql('csv_data', conn, if_exists='replace', index=False)
#Check table
pd.read_sql('select * from csv_data limit 10', conn)
#Check number of records in table
pd.read_sql('select count(*) from csv_data', conn)

,Rank,Player,REC,TGT,REC_YDS,Y/R,LG,20+,REC_TD,ATT,YDS.1,RUSH_TD,FL,G,FPTS,FPTS/G,OWN,Team
0,1.0,Michael Thomas,149.0,185.0,"1,725",11.6,49.0,17.0,9.0,1.0,-9.0,0.0,0.0,16.0,225.6,14.1,100.0%,NO
1,2.0,Chris Godwin,86.0,121.0,"1,333",15.5,71.0,25.0,9.0,1.0,8.0,0.0,0.0,14.0,190.1,13.6,99.9%,TB
2,3.0,Kenny Golladay,65.0,116.0,"1,190",18.3,75.0,22.0,11.0,0.0,0.0,0.0,1.0,16.0,183.0,11.4,99.1%,DET
3,4.0,Cooper Kupp,94.0,134.0,"1,161",12.4,66.0,21.0,10.0,2.0,4.0,0.0,0.0,16.0,176.5,11.0,97.2%,LAR
4,5.0,Julio Jones,99.0,157.0,"1,394",14.1,54.0,21.0,6.0,2.0,-3.0,0.0,0.0,15.0,175.1,11.7,100.0%,ATL
5,6.0,DeVante Parker,72.0,128.0,"1,202",16.7,51.0,21.0,9.0,0.0,0.0,0.0,0.0,16.0,174.2,10.9,95.0%,MIA
6,7.0,Amari Cooper,79.0,119.0,"1,189",15.1,53.0,17.0,8.0,1.0,6.0,0.0,0.0,16.0,167.5,10.5,99.3%,DAL
7,8.0,Mike Evans,67.0,118.0,"1,157",17.3,67.0,17.0,8.0,0.0,0.0,0.0,0.0,13.0,165.7,12.7,99.9%,TB
8,9.0,DeAndre Hopkins,104.0,150.0,"1,165",11.2,43.0,16.0,7.0,2.0,18.0,0.0,0.0,15.0,165.5,11.0,100.0%,ARI
9,10.0,A.J. Brown,52.0,84.0,"1,051",20.2,91.0,15.0,8.0,3.0,60.0,1.0,0.0,16.0,165.1,10.3,97.3%,TEN


In [9]:
#Check number of records in table
pd.read_sql('select count(*) from csv_data', conn)

,count(*)
0,525


In [22]:
#Connect to SQL Lite Database
#Use this connection statement below if you want to export after disconnecting or reloading Notebook.
#conn = sqlite3.connect('DSC540_final_db.sqlite')

#Export connection to csv
db_df = pd.read_sql_query("select * from csv_data", conn)
db_df.to_csv('csv_data_export.csv', index=False)

In [23]:
#Close database connnection
conn.close()

# End of CSV process

# Step 2) Web Scraping Data

## Import Web Scraping modules

In [10]:
# Import scraping modules
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Import data manipulation modules
import pandas as pd
import numpy as np
import sqlite3


In [11]:
# URL of page
url = 'https://www.pro-football-reference.com/years/2019/receiving.htm'
# Open URL and pass to BeautifulSoup
html = urlopen(url)
stats_page = BeautifulSoup(html)

In [12]:
# Collect table headers
column_headers = stats_page.findAll('tr')[0]
column_headers = [i.getText() for i in column_headers.findAll('th')]

In [13]:
# Collect table rows
rows = stats_page.findAll('tr')[1:]

# Get stats from each row
wr_stats = []
for i in range(len(rows)):
    wr_stats.append([col.getText() for col in rows[i].findAll('td')])

In [14]:
# Create DataFrame from our scraped data
wr_scrape_data = pd.DataFrame(wr_stats, columns=column_headers[1:])

In [15]:
wr_scrape_data.head()

,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Michael Thomas*+,NOR,26,WR,16,15,185,149,80.5%,1725,11.6,9,91,49,9.3,9.3,107.8,1
1,Christian McCaffrey *+,CAR,23,RB,16,16,142,116,81.7%,1005,8.7,4,58,28,7.1,7.3,62.8,1
2,Keenan Allen*,LAC,27,WR,16,16,149,104,69.8%,1199,11.5,6,63,45,8.0,6.5,74.9,0
3,DeAndre Hopkins*+,HOU,27,WR,15,15,150,104,69.3%,1165,11.2,7,68,43,7.8,6.9,77.7,0
4,Julian Edelman,NWE,33,WR,16,13,153,100,65.4%,1117,11.2,6,54,44,7.3,6.3,69.8,3


In [16]:
# Remove special characters

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–", "+"]

for char in spec_chars:
    wr_scrape_data['Player'] = wr_scrape_data['Player'].str.replace(char, ' ')
wr_scrape_data.head()

,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Michael Thomas,NOR,26,WR,16,15,185,149,80.5%,1725,11.6,9,91,49,9.3,9.3,107.8,1
1,Christian McCaffrey,CAR,23,RB,16,16,142,116,81.7%,1005,8.7,4,58,28,7.1,7.3,62.8,1
2,Keenan Allen,LAC,27,WR,16,16,149,104,69.8%,1199,11.5,6,63,45,8.0,6.5,74.9,0
3,DeAndre Hopkins,HOU,27,WR,15,15,150,104,69.3%,1165,11.2,7,68,43,7.8,6.9,77.7,0
4,Julian Edelman,NWE,33,WR,16,13,153,100,65.4%,1117,11.2,6,54,44,7.3,6.3,69.8,3


In [17]:
# Remove Pct sign from Ctch% output
wr_scrape_data['Ctch%'] = wr_scrape_data['Ctch%'].str.replace('%', '')
wr_scrape_data.head()

,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Michael Thomas,NOR,26,WR,16,15,185,149,80.5,1725,11.6,9,91,49,9.3,9.3,107.8,1
1,Christian McCaffrey,CAR,23,RB,16,16,142,116,81.7,1005,8.7,4,58,28,7.1,7.3,62.8,1
2,Keenan Allen,LAC,27,WR,16,16,149,104,69.8,1199,11.5,6,63,45,8.0,6.5,74.9,0
3,DeAndre Hopkins,HOU,27,WR,15,15,150,104,69.3,1165,11.2,7,68,43,7.8,6.9,77.7,0
4,Julian Edelman,NWE,33,WR,16,13,153,100,65.4,1117,11.2,6,54,44,7.3,6.3,69.8,3


In [18]:
# Select stat categories
categories = ['Player', 'Tm', 'Age', 'Pos', 'GS', 'Ctch%', 'Age_Rank', 'Ctch%_Rank']


In [19]:
#2) Create new columns with percentile rank
for i in categories:
    wr_scrape_data[i + '_Rank'] = wr_scrape_data[i].rank(pct=True)


In [20]:
# Create data subset for analysis
wr_scrape_subset = wr_scrape_data[categories]
wr_scrape_subset.head()

,Player,Tm,Age,Pos,GS,Ctch%,Age_Rank,Ctch%_Rank
0,Michael Thomas,NOR,26,WR,15,80.5,0.613035,0.914460
1,Christian McCaffrey,CAR,23,RB,16,81.7,0.162933,0.924644
2,Keenan Allen,LAC,27,WR,16,69.8,0.738289,0.695519
3,DeAndre Hopkins,HOU,27,WR,15,69.3,0.738289,0.686354
4,Julian Edelman,NWE,33,WR,13,65.4,0.972505,0.543788


In [21]:
#3) Update additional field names
wr_scrape_subset.rename(columns = {'Tm':'Team'}, inplace = True) 
wr_scrape_subset.rename(columns = {'Ctch%':'catch_pct'}, inplace = True) 
wr_scrape_subset.rename(columns = {'Ctch%_Rank':'catch_pct_rank'}, inplace = True) 
wr_scrape_subset.head()


C:\Users\Dan\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Player,Team,Age,Pos,GS,catch_pct,Age_Rank,catch_pct_rank
0,Michael Thomas,NOR,26,WR,15,80.5,0.613035,0.914460
1,Christian McCaffrey,CAR,23,RB,16,81.7,0.162933,0.924644
2,Keenan Allen,LAC,27,WR,16,69.8,0.738289,0.695519
3,DeAndre Hopkins,HOU,27,WR,15,69.3,0.738289,0.686354
4,Julian Edelman,NWE,33,WR,13,65.4,0.972505,0.543788


In [22]:
# drop all rows with any NaN and None values
wr_scrape_final = wr_scrape_subset.dropna()
wr_scrape_final.head()

,Player,Team,Age,Pos,GS,catch_pct,Age_Rank,catch_pct_rank
0,Michael Thomas,NOR,26,WR,15,80.5,0.613035,0.914460
1,Christian McCaffrey,CAR,23,RB,16,81.7,0.162933,0.924644
2,Keenan Allen,LAC,27,WR,16,69.8,0.738289,0.695519
3,DeAndre Hopkins,HOU,27,WR,15,69.3,0.738289,0.686354
4,Julian Edelman,NWE,33,WR,13,65.4,0.972505,0.543788


In [46]:
#Connect to SQL Lite Database
conn = sqlite3.connect('DSC540_final_db.sqlite')
#Insert Web Scraping dataframe
wr_scrape_final.to_sql('web_data', conn, if_exists='replace', index=False)
#Check table
pd.read_sql('select * from web_data limit 5', conn)

,Player,Team,Age,Pos,GS,catch_pct,Age_Rank,catch_pct_rank
0,Michael Thomas,NOR,26,WR,15,80.5,0.613035,0.914460
1,Christian McCaffrey,CAR,23,RB,16,81.7,0.162933,0.924644
2,Keenan Allen,LAC,27,WR,16,69.8,0.738289,0.695519
3,DeAndre Hopkins,HOU,27,WR,15,69.3,0.738289,0.686354
4,Julian Edelman,NWE,33,WR,13,65.4,0.972505,0.543788


In [23]:
#Count number of records

pd.read_sql('select count(*) from web_data', conn)

,count(*)
0,491


In [17]:
#Connect to SQL Lite Database
#Use this connection statement below if you want to export after disconnecting or reloading Notebook.
#conn = sqlite3.connect('DSC540_final_db.sqlite')

#Export connection to csv
db_df = pd.read_sql_query("select * from web_data", conn)
db_df.to_csv('web_data_export.csv', index=False)

In [47]:
#Close database connnection
conn.close()

## Join csv and web scraping data into sql lite table


In [25]:
#Connect to SQL Lite Database
#Use this connection statement below if you want to export after disconnecting or reloading Notebook.
conn = sqlite3.connect('DSC540_final_db.sqlite')

#Export connection to csv
sqllite_join = pd.read_sql_query("select csv_data.Player, csv_data.fpts, web_data.Age, web_data.catch_pct  from web_data join csv_data on web_data.Player = csv_data.Player", conn)
#Insert Web Scraping dataframe
sqllite_join.to_sql('web_csv_data_join', conn, if_exists='replace', index=False)
#Check table
pd.read_sql('select * from web_csv_data_join limit 5', conn)



,Player,FPTS,Age,catch_pct
0,Keenan Allen,157.5,27,69.8
1,Chris Godwin,190.1,23,71.1
2,Davante Adams,129.7,27,65.4
3,Jarvis Landry,154.4,27,60.1
4,Amari Cooper,167.5,25,66.4


# End Web Scraping process

# 3) API Process

## Get API modules

In [26]:
#Import Packages
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import sqlite3


In [27]:
#Get API data
url = "https://www.fantasyfootballdatapros.com/api/projections"
JSONContent = requests.get(url).json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True)
print(content)

[
    {
        "player_name": "Lamar Jackson",
        "pos": "QB",
        "projection": 353.41,
        "team": "BAL"
    },
    {
        "player_name": "Christian McCaffrey",
        "pos": "RB",
        "projection": 338.52,
        "team": "CAR"
    },
    {
        "player_name": "Patrick Mahomes",
        "pos": "QB",
        "projection": 331.01,
        "team": "KC"
    },
    {
        "player_name": "Deshaun Watson",
        "pos": "QB",
        "projection": 307.07,
        "team": "HOU"
    },
    {
        "player_name": "Michael Thomas",
        "pos": "WR",
        "projection": 300.99,
        "team": "NO"
    },
    {
        "player_name": "Dalvin Cook",
        "pos": "RB",
        "projection": 299.89,
        "team": "MIN"
    },
    {
        "player_name": "Dak Prescott",
        "pos": "QB",
        "projection": 298.73,
        "team": "DAL"
    },
    {
        "player_name": "Russell Wilson",
        "pos": "QB",
        "projection": 296.8,
        "team"

In [28]:
#Get API data step 1
r = requests.get('https://www.fantasyfootballdatapros.com/api/projections')
data = r.text
api_data = pd.read_json(data, orient='records')

In [29]:
#Get api data into data frame
api_data.head(5)

,player_name,pos,projection,team
0,Lamar Jackson,QB,353.41,BAL
1,Christian McCaffrey,RB,338.52,CAR
2,Patrick Mahomes,QB,331.01,KC
3,Deshaun Watson,QB,307.07,HOU
4,Michael Thomas,WR,300.99,NO


In [30]:
#Subset the data to WR projections only
api_data_subset_wr = api_data[api_data["pos"] == 'WR']
api_data_subset_wr.head()

,player_name,pos,projection,team
4,Michael Thomas,WR,300.99,NO
16,DeAndre Hopkins,WR,268.42,ARI
20,Julio Jones,WR,262.85,ATL
22,Davante Adams,WR,261.75,GB
23,Chris Godwin,WR,260.47,TB


In [31]:
#Remove any special characters from player name
# initializing bad_chars_list 

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

for char in spec_chars:
    api_data_subset_wr['player_name'] = api_data_subset_wr['player_name'].str.replace(char, ' ')
api_data_subset_wr

C:\Users\Dan\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,player_name,pos,projection,team
4,Michael Thomas,WR,300.99,NO
16,DeAndre Hopkins,WR,268.42,ARI
20,Julio Jones,WR,262.85,ATL
22,Davante Adams,WR,261.75,GB
23,Chris Godwin,WR,260.47,TB
...,...,...,...,...
175,DaeSean Hamilton,WR,125.01,DEN
176,Malcolm Brown,WR,124.55,LAR
178,Allen Lazard,WR,123.85,GB
180,Corey Davis,WR,123.06,TEN


In [33]:
#Connect to SQL Lite Database
conn = sqlite3.connect('DSC540_final_db.sqlite')
#Insert Web Scraping dataframe
api_data_subset_wr.to_sql('api_data', conn, if_exists='replace', index=False)
#Check table
pd.read_sql('select * from api_data limit 5', conn)

,player_name,pos,projection,team
0,Michael Thomas,WR,300.99,NO
1,DeAndre Hopkins,WR,268.42,ARI
2,Julio Jones,WR,262.85,ATL
3,Davante Adams,WR,261.75,GB
4,Chris Godwin,WR,260.47,TB


In [63]:
#Connect to SQL Lite Database
#Use this connection statement below if you want to export after disconnecting or reloading Notebook.
#conn = sqlite3.connect('DSC540_final_db.sqlite')

#Export connection to csv
db_df = pd.read_sql_query("select * from api_data", conn)
db_df.to_csv('api_data_export.csv', index=False)

## Combine csv and api data

In [46]:
#Connect to SQL Lite Database
#Use this connection statement below if you want to export after disconnecting or reloading Notebook.
conn = sqlite3.connect('DSC540_final_db.sqlite')

#Export connection to csv
sqllite_join2 = pd.read_sql_query("select csv_data.Player, csv_data.REC, csv_data.REC_YDS, csv_data.FPTS, api_data.projection, api_data.Team  from csv_data join api_data on api_data.player_name = TRIM(csv_data.Player)", conn)
#Insert Web Scraping dataframe
sqllite_join2.to_sql('api_csv_data_join', conn, if_exists='replace', index=False)
#Check table
pd.read_sql('select * from api_csv_data_join limit 10', conn)

sqllite_join2.head(5)

#pd.read_sql('select csv_data.Player, csv_data.REC, csv_data.REC_YDS, csv_data.FPTS, api_data.projection, api_data.Team  from csv_data left join api_data on api_data.player_name = csv_data.Player', conn)



,Player,REC,REC_YDS,FPTS,projection,team
0,Michael Thomas,149.0,"1,725",225.6,300.99,NO
1,Chris Godwin,86.0,"1,333",190.1,260.47,TB
2,Kenny Golladay,65.0,"1,190",183.0,227.23,DET
3,Cooper Kupp,94.0,"1,161",176.5,215.76,LAR
4,Julio Jones,99.0,"1,394",175.1,262.85,ATL


In [47]:
#Close database connnection
conn.close()

# End of API Data process